## DIA 054: Creacion de un Sistema de Webhooks en Flask para Eventos en Tiempo Real

In [ ]:
Hoy implementaremos Webhooks en Flask, lo que permitirá que nuestra API reciba eventos en tiempo real desde otros sistemas o servicios.

✅ ¿Qué es un Webhook?
Un Webhook es un endpoint especial que recibe notificaciones automáticas de otros sistemas cuando ocurre un evento específico.

✅ Ejemplo de Uso

Procesar pagos en tiempo real desde Stripe o PayPal.
Actualizar registros automáticamente cuando un usuario modifica su perfil en otro sistema.
Recibir alertas cuando un servicio externo genera un evento.
🖥️ Código Completo (webhooks.py)
python
Copiar
Editar
import os
import json
import logging
from datetime import datetime
from flask import Flask, request, jsonify

# Configuración básica de Flask
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Simulación de Base de Datos
# ---------------------------
EVENTS_LOG = []  # Almacena eventos recibidos temporalmente

# ---------------------------
# Endpoint para Webhooks
# ---------------------------
@app.route('/webhook', methods=['POST'])
def webhook():
    """
    Recibe eventos desde un sistema externo.
    """
    try:
        event = request.json
        if not event:
            return jsonify({"error": "No se recibió JSON"}), 400
        
        event_type = event.get("type")
        timestamp = datetime.utcnow().isoformat()
        
        logger.info(f"📩 Evento recibido: {event_type} - {event}")
        
        # Guardar el evento en memoria
        EVENTS_LOG.append({"type": event_type, "data": event, "timestamp": timestamp})

        return jsonify({"message": "Evento recibido", "type": event_type}), 200

    except Exception as e:
        logger.error(f"⚠️ Error procesando el Webhook: {e}")
        return jsonify({"error": "Error interno"}), 500

# ---------------------------
# Endpoint para Ver Eventos Recibidos
# ---------------------------
@app.route('/events', methods=['GET'])
def get_events():
    """
    Devuelve una lista de eventos recibidos.
    """
    return jsonify(EVENTS_LOG), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True, port=5001)
📡 Simulación de Envío de un Webhook con cURL
Para probar el Webhook, podemos usar cURL o Postman:

sh
Copiar
Editar
curl -X POST http://127.0.0.1:5001/webhook \
     -H "Content-Type: application/json" \
     -d '{
           "type": "payment_success",
           "user_id": 12345,
           "amount": 19.99,
           "currency": "USD"
         }'
📩 Respuesta esperada:

json
Copiar
Editar
{
    "message": "Evento recibido",
    "type": "payment_success"
}
Luego, podemos verificar los eventos almacenados con:

sh
Copiar
Editar
curl -X GET http://127.0.0.1:5001/events
🔍 Explicación de las Principales Implementaciones
🔹 📡 Webhooks en Flask (/webhook)

Recibe datos JSON desde un sistema externo.
Extrae el tipo de evento (type).
Registra el evento en un log temporal (EVENTS_LOG).
🔹 📜 Almacenamiento Temporal de Eventos

Se usa una lista en memoria para almacenar los eventos recibidos.
Permite consultar los eventos con el endpoint /events.
🔹 🔍 Logging de Eventos en Tiempo Real

Los eventos recibidos se imprimen en consola con logger.info().
Si hay un error, se captura y registra con logger.error().
🔹 ✅ Seguridad Básica en Webhooks

Se valida que el request tenga datos JSON válidos.
Se evita el acceso no autorizado con validaciones futuras.